In [ ]:
from utils import *

NOTEBOOK_ID = '22'
RUN_ESM = False
SAVE_IDXS = False
RUN_MODELS = False

# encode new datasets

In [ ]:
if RUN_ESM:
    morffy_df = pd.read_csv(f'{COMPARE_MORFFY_DIR}/01-dataset/TrainingsData.csv').drop_duplicates(subset='Sequence',keep=False)
    morffy_df = morffy_df[morffy_df['Sequence'].apply(lambda x: len(x) == 40)].rename(columns={'Sequence':'AAseq'})

    model = 'esm2_t33_650M_UR50D'
    layer = 33
    tmp_df = morffy_df.copy()
    out_faa = f'{COMPARE_MORFFY_DIR}/01-dataset/morffy.faa'
    embeddings_dir = f'{COMPARE_MORFFY_DIR}/01-dataset/morffy_{model}'
    out_pkl = f'{COMPARE_MORFFY_DIR}/01-dataset/morffy_{model}-layer{layer}-representations.pkl'

    tmp_df['ID'] = [f'morffy-{str(i).zfill(6)}' for i in range(len(tmp_df))]
    df_to_fasta(tmp_df,'ID','AAseq',out_faa)
    if len(glob(f'{embeddings_dir}/*')) == 0:
        print('running ESM')
        run_esm(out_faa,
                embeddings_dir,
                model=model,
                layer=layer)
    if not os.path.isfile(out_pkl):
        print('saving representations')
        embedding_df = tmp_df.copy()
        embedding_df[model] = embedding_df['ID'].apply(lambda x: get_embedding(embeddings_dir,x,layer=layer))
        embedding_df.to_pickle(out_pkl)
        del tmp_df

if RUN_ESM:
    harmonized_df = pd.read_csv(f'{HARMONIZE_DIR}/04-results/new_to_old_harmonized_dataset.csv').query('dataset != "Mycocosm_Overlap"').drop_duplicates(subset='AAseq',keep=False)
    harmonized_df = harmonized_df[['dataset','AAseq','linear_harmonized_activity_scaled','linear_harmonized_activity']]

    tmp_df = harmonized_df.copy()
    tmp_df['ID'] = [f'harmonized-{str(i).zfill(5)}' for i in range(len(tmp_df))]
    model = 'esm2_t33_650M_UR50D'
    layer = 33

    df_to_fasta(tmp_df,'ID','AAseq','../../02-OUTPUT/20-sandbox/harmonized.faa')
    print('running ESM')
    run_esm(f'../../02-OUTPUT/20-sandbox//harmonized.faa',
            f'../../02-OUTPUT/20-sandbox/{model}',
            model=model,
            layer=layer)
    print('saving representations')
    embedding_df = tmp_df.copy()
    embedding_df[model] = embedding_df['ID'].apply(lambda x: get_embedding(f'../../02-OUTPUT/20-sandbox/{model}/',x,layer=layer))
    embedding_df.to_pickle(f'../../02-OUTPUT/20-sandbox/harmonized_{model}-layer{layer}-representations.pkl')
    del tmp_df

# standardize datasets for comparison

In [ ]:
if not os.path.isfile(f'{COMPARE_DIR}/morffy_dataset.pkl'):
    datasets = {}
    hummel_df = pd.read_pickle(f'{ENCODING_DIR}/01-dataset/hummel_encodings.pkl').drop_duplicates(subset='AAseq',keep=False)
    onehot_X = np.asarray([np.array(emb) for emb in hummel_df['onehot_encoding']])
    alphabet_X = np.asarray([[AA_TO_I[aa] for aa in x] for x in hummel_df['AAseq']])
    hummel_df = pd.read_pickle(f'{EMBEDDING_DIR}/01-dataset/esm2_t33_650M_UR50D-layer33-representations.pkl').drop_duplicates(subset='AAseq',keep=False)
    esm_X = np.asarray([np.array(emb) for emb in hummel_df['esm2_t33_650M_UR50D']])
    y = hummel_df['activity'].to_numpy()
    y_cont = y.reshape(-1, 1)
    y_cont = preprocessing.MinMaxScaler().fit_transform(y_cont)
    y_cont = preprocessing.StandardScaler().fit_transform(y_cont)
    thresh = y_cont.mean() + y_cont.std() 
    y_bin = (y_cont >= thresh).astype(np.int64).reshape(-1, 1)
    datasets['hummel'] = {'onehot':onehot_X, 
                          'onehot_flat':onehot_X.reshape(onehot_X.shape[0],onehot_X.shape[1]*onehot_X.shape[2]),
                          'alphabet':alphabet_X, 
                          'esm':esm_X,
                          'esm_mean':esm_X.mean(axis=-1),
                          'y_cont':y_cont,
                          'y_bin':y_bin}
    with open(f'{COMPARE_DIR}/hummel_dataset.pkl', 'wb') as f:
        pickle.dump(datasets, f)

    datasets = {}
    harmonized_df = pd.read_pickle(f'{HARMONIZE_DIR}/04-results/esm2_t33_650M_UR50D-layer33-representations.pkl').query('dataset != "Mycocosm_Overlap"').drop_duplicates(subset='AAseq',keep=False)
    onehot_X = np.asarray([np.array(emb) for emb in harmonized_df['AAseq'].apply(lambda x: one_hot_encode(x))])
    alphabet_X = np.asarray([[AA_TO_I[aa] for aa in x] for x in harmonized_df['AAseq']])
    esm_X = np.asarray([np.array(emb) for emb in harmonized_df['esm2_t33_650M_UR50D']])
    y = harmonized_df['linear_harmonized_activity'].to_numpy()
    y_cont = y.reshape(-1, 1)
    y_cont = preprocessing.MinMaxScaler().fit_transform(y_cont)
    y_cont = preprocessing.StandardScaler().fit_transform(y_cont)
    thresh = y_cont.mean() + y_cont.std() 
    y_bin = (y_cont >= thresh).astype(np.int64).reshape(-1, 1)

    datasets['harmonized'] = {'onehot':onehot_X, 
                              'onehot_flat':onehot_X.reshape(onehot_X.shape[0],onehot_X.shape[1]*onehot_X.shape[2]),
                              'alphabet':alphabet_X, 
                              'esm':esm_X,
                              'esm_mean':esm_X.mean(axis=-1),
                              'y_cont':y_cont,
                              'y_bin':y_bin}
    with open(f'{COMPARE_DIR}/harmonized_dataset.pkl', 'wb') as f:
        pickle.dump(datasets, f)

    datasets = {}
    morffy_df = pd.read_csv(f'{MORFFY_DIR}/01-dataset/TrainingsData.csv').drop_duplicates(subset='Sequence',keep=False)
    morffy_df = morffy_df[morffy_df['Sequence'].apply(lambda x: len(x) == 40)].rename(columns={'Sequence':'AAseq'})
    onehot_X = np.asarray([np.array(emb) for emb in morffy_df['AAseq'].apply(lambda x: one_hot_encode(x))])
    alphabet_X = np.asarray([[AA_TO_I[aa] for aa in x] for x in morffy_df['AAseq']])
    morffy_df = pd.read_pickle(f'{MORFFY_DIR}/01-dataset/morffy_esm2_t33_650M_UR50D-layer33-representations.pkl').drop_duplicates(subset='AAseq',keep=False)
    morffy_df = morffy_df[morffy_df['AAseq'].apply(lambda x: len(x) == 40)]
    esm_X = np.asarray([np.array(emb) for emb in morffy_df['esm2_t33_650M_UR50D']])
    y = morffy_df['Score'].to_numpy()
    y_cont = y.reshape(-1, 1)
    y_cont = preprocessing.MinMaxScaler().fit_transform(y_cont)
    y_cont = preprocessing.StandardScaler().fit_transform(y_cont)
    thresh = y_cont.mean() + y_cont.std() 
    y_bin = (y_cont >= thresh).astype(np.int64).reshape(-1, 1)
    datasets['morffy'] = {'onehot':onehot_X,
                          'onehot_flat':onehot_X.reshape(onehot_X.shape[0],onehot_X.shape[1]*onehot_X.shape[2]),
                          'alphabet':alphabet_X, 
                          'esm':esm_X,
                          'esm_mean':esm_X.mean(axis=-1),
                          'y_cont':y_cont,
                          'y_bin':y_bin}
    with open(f'{COMPARE_DIR}/morffy_dataset.pkl', 'wb') as f:
        pickle.dump(datasets, f)

# standardize held-out test splits

In [ ]:
if SAVE_IDXS:
    for dataset_path in glob(f'{COMPARE_DIR}/*dataset.pkl'):
        dataset = pd.read_pickle(dataset_path)
        for dataset_name, data in dataset.items():
            X, y_cont, y_bin = data['esm'], data['y_cont'], data['y_bin']
            print(dataset_name)
            for state in range(1,4):
                threshold = np.median(y_cont)
                _, idx = split_dataset(X,y_cont,threshold,scaler='standard',random_state=state)
                with open(f'{ADHUNTER_DIR}/03-comparison_idxs/{dataset_name}_esm_dataset_idx_state{state}.pkl', 'wb') as f:
                    pickle.dump(idx, f)

# evaluate ADhunter

In [ ]:
query_df = pd.read_csv(f'{HAMMING_ENSEMBLE_DIR}/03-results/top_models.csv')
df_filtered = pd.read_csv(f'{HAMMING_ENSEMBLE_DIR}/03-results/model_testing_filtered.csv').set_index('file')

if RUN_MODELS:
    for dataset_path in glob(f'{COMPARE_DIR}/*dataset.pkl'):
        dataset = pd.read_pickle(dataset_path)
        for dataset_name, data in dataset.items():
            if dataset_name == 'hummel':
                continue
            X, y_cont, y_bin = data['esm'], data['y_cont'], data['y_bin']
            print(dataset_name)
            for state in range(1,4):
                print(state)
                threshold = np.median(y_cont)
                dataset, idx = split_dataset(X,y_cont,threshold,scaler='standard',random_state=state)
            
                train, val, _ = dataset
                (X_train, _, y_cont_train) = train
                (X_val, _, y_cont_val) = val

                out_dir = f'{ADHUNTER_DIR}/02-ensemble'
                _, selected_indices = select_maximally_different_arrays(df_filtered.to_numpy(), 20)
                hits_df = df_filtered.reset_index()
                hits_df = hits_df.loc[selected_indices].set_index('file')
                for idx, row in query_df[query_df.file.isin(hits_df.index)].reset_index(drop=True).iterrows():
                    print(f'==================={idx/20}===================')
                    train_ds = TensorDataset(X_train.to(torch.float), y_cont_train.to(torch.float))
                    val_ds = TensorDataset(X_val.to(torch.float), y_cont_val.to(torch.float))
                    train_dl = DataLoader(train_ds, batch_size=row['batch_size'], shuffle=False)
                    val_dl = DataLoader(val_ds, batch_size=row['batch_size'], shuffle=False)

                    model = ADhunterSystem_v2(
                        embedding_size=X_train[0].shape[1],
                        hidden=row['hidden_size'], 
                        kernel_size=row['kernel_size'], 
                        dilation=row['dilation'], 
                        num_res_blocks=row['num_res_blocks'],
                        seq_len=X_train[0].shape[0]
                        )

                    OUTPUT_FILE = f"{dataset_name}_ADhunter_v2_state{state}_h{row['hidden_size']}_k{row['kernel_size']}_d{row['dilation']}_r{row['num_res_blocks']}_b{row['batch_size']}"
                    csv_logger = CSVLogger(f"{out_dir}/01-logs",name=OUTPUT_FILE,version='')
                    checkpoint_callback = ModelCheckpoint(dirpath=f"{out_dir}/02-models", monitor="val_loss", filename=OUTPUT_FILE, save_last=False)
                    early_stopping = EarlyStopping('val_loss', patience=PATIENCE)
                    trainer = pl.Trainer(accelerator='gpu', devices=1, callbacks=[checkpoint_callback, early_stopping], logger=[csv_logger], max_epochs=MAX_EPOCHS)
                    trainer.fit(model, train_dataloaders=train_dl, val_dataloaders=val_dl)

                    model = model.load_from_checkpoint(checkpoint_callback.best_model_path)
                    torch.save(model.cpu().state_dict(), f'{out_dir}/02-models/{OUTPUT_FILE}.pt')

In [ ]:
if RUN_MODELS:
    ensemble_tests = {}
    ensemble_metrics = {}
    for dataset_path in glob(f'{COMPARE_DIR}/*dataset.pkl'):
        dataset = pd.read_pickle(dataset_path)
        for dataset_name, data in dataset.items():
            if dataset_name == 'hummel':
                continue
            X, y_cont, y_bin = data['esm'], data['y_cont'], data['y_bin']
            print(dataset_name)
            for state in range(1,4):
                threshold = np.median(y_cont)
                dataset, idx = split_dataset(X,y_cont,threshold,scaler='standard',random_state=state)
                _, _, test = dataset
                (X_test, y_bin_test, y_test) = test
                y_bin_test = y_bin_test.reshape(-1).numpy()

                predictions, variances = get_new_uncertainty(dataset_name, state, X_test)
                ensemble_tests[f'{dataset_name}_{state}'] = [predictions, variances, y_bin_test, y_test.numpy().reshape(-1)]
                ensemble_metrics[f'{dataset_name}_{state}'] = [
                    pearsonr(y_test.numpy().reshape(-1),predictions),
                    spearmanr(y_test.numpy().reshape(-1),predictions),
                    mean_squared_error(y_test.numpy().reshape(-1),predictions,squared=False),
                ]
    ensemble_tests = pd.DataFrame(ensemble_tests).T.reset_index()
    ensemble_tests.to_pickle(f'{ADHUNTER_DIR}/02-ensemble/03-results/ensemble_test.pkl')
    ensemble_metrics = pd.DataFrame(ensemble_metrics).T.reset_index()
    ensemble_metrics.to_pickle(f'{ADHUNTER_DIR}/02-ensemble/03-results/ensemble_metrics.pkl')
else:
    ensemble_tests = pd.read_pickle(f'{ADHUNTER_DIR}/02-ensemble/03-results/ensemble_test.pkl')
    ensemble_metrics = pd.read_pickle(f'{ADHUNTER_DIR}/02-ensemble/03-results/ensemble_metrics.pkl')
ensemble_metrics['state'] = ensemble_metrics['index'].apply(lambda x: x.split('_')[-1])
ensemble_metrics['dataset'] = ensemble_metrics['index'].apply(lambda x: x.split('_')[0])
ensemble_metrics['r'] = ensemble_metrics[0].apply(lambda x: x[0])
ensemble_metrics['rmse'] = ensemble_metrics[2]
ensemble_metrics.drop(columns=['index',0,1,2,'state']).groupby('dataset').mean().style


,r,rmse
dataset,,
harmonized,0.817676,0.576275
morffy,0.681990,0.739541


In [ ]:
ensemble_tests.columns = ['params','predictions','uncertainty','y_bin_test','y_test']
classification_metrics_dict = {}
for idx,row in ensemble_tests.iterrows():
    dataset = row['params']
    y_test_hat = row['predictions']
    y_test = row['y_test']
    y_test_hat_bin = (y_test_hat >= 1.0).astype(int)
    y_test_bin = (y_test >= 1.0).astype(int)
    classification_metrics_dict[dataset] = classification_metrics(y_test_hat_bin,y_test_bin)

classification_df = pd.DataFrame(classification_metrics_dict).T.reset_index()
classification_df['state'] = classification_df['index'].apply(lambda x: x.split('_')[-1])
classification_df['dataset'] = classification_df['index'].apply(lambda x: x.split('_')[0])
classification_df.drop(columns=['index','state']).groupby('dataset').mean().style

,Accuracy,F1 Score
dataset,,
harmonized,0.905041,0.909919
morffy,0.931724,0.935101
